In [1]:
### import necessary libraries
import ee
import geemap
import sklearn
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [12]:
# Assuming you've already authenticated and initialized Earth Engine
taluks = ee.FeatureCollection("users/ujavalgandhi/gee-water-resources/kgis_taluks")
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')

year = 2019
startDate = ee.Date.fromYMD(year=year, month=1, day=1)  # Use keyword arguments for clarity
endDate = startDate.advance(1, 'year')

Map = geemap.Map()

filtered = chirps.filter(ee.Filter.date(start=startDate, end=endDate))

# Calculate the total rainfall for the year
total = filtered.reduce(reducer=ee.Reducer.sum())

# Display the total rainfall image.
palette = ['#ffffcc', '#a1dab4', '#41b6c4', '#2c7fb8', '#253494']
visParams = {
    'min': 0,
    'max': 2000,
    'palette': palette
}

Map.centerObject(ee_object=taluks)
Map.addLayer(total, vis_params=visParams, name='Total Precipitation')
Map.addLayer(taluks, vis_params={'color': 'grey'}, name='Taluks')


# reduceRegion()
results = total.reduceRegions(
    collection = taluks,
    reducer=ee.Reducer.mean(),
    scale=5566,
    )


info = results.first().getInfo()
#print(json.dumps(info, indent=2))




Map.addLayer(results, vis_params={'color': 'red'}, name='Mean Precipitation')
display(Map)


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [13]:
# Select the columns and rename them
exportCol = results.select(['KGISTalukN', 'mean'], ['taluk', 'average_rainfall'])

# Export the exportCol as a CSV
task = ee.batch.Export.table.toDrive(
    collection=exportCol,
    description='Average Rainfall CSV',
    fileFormat='CSV',
    selectors=['taluk', 'average_rainfall']
)

# Start the export task
task.start()